In [1]:
from pathlib import Path
import csv
import itertools
import numpy as np
from pprint import PrettyPrinter


import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

import h5py

import sklearn
import sklearn.metrics

pprint = PrettyPrinter()

## Configuration

In [2]:
SELECTED_LANGUAGES = {'_language_independent', 'francais', 'maninka', 'pular', 'susu'}

BASE_DIR = Path('/media/xtrem/data/experiments/nicolingua-0002-va-asr/datasets/gn_va_asr_dataset_2020-08-24_02')


ANNOTATIONS_PATH = BASE_DIR/ "annotated_segments" / "metadata.csv"


FEATURE_NAMES = [
    "wav2vec_features-c", 
    "wav2vec_features-z", 
    "retrained-wav2vec_features-c", 
    "retrained-wav2vec_features-z"
]

# CONV_POOLING_TYPES = ['avg', 'max']
CONV_POOLING_TYPES = ['avg']
OBJECTIVE_TYPES = ['voice_cmd', 'voice_cmd__and__voice_cmd_lng']
CONV_DROPOUT_PROBABILITIES = [0.2]
FC_DROPOUT_PROBABILITIES = [0.2]


TRAIN_PERCENT = .7
FOLD_COUNT = 5


RESULTS_DIR = f'results_101'
EPOCHS = 1000
BATCH_SIZE = 512
MAX_FEATURE_SEQUENCE_LENGTH = 200

GPU_ID = 0
device = torch.device(f"cuda:{GPU_ID}")

## Load & shuffle metadata records

In [3]:
def load_metadata():
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for r in reader:
            if r['language'] in SELECTED_LANGUAGES:
                yield r

In [4]:
# load
metadata_records = list(load_metadata())

# shuffle
metadata_shuffler_rs = np.random.RandomState(seed=42)
metadata_shuffler_rs.shuffle(metadata_records)

In [5]:
bias_category_fields = [
     "device_id"
    ,"language"
    ,"speaker_gender"
    ,"speaker_mothertongue"
]

bias_categories = {}
for c in bias_category_fields:
    bias_categories[c] = sorted({r[c] for r in metadata_records})


_ = [print(f"\n{k}: \n\t{','.join(v)}") for k,v in bias_categories.items()]


device_id: 
	d001,d002,d003

language: 
	_language_independent,francais,maninka,pular,susu

speaker_gender: 
	F,M

speaker_mothertongue: 
	maninka,pular,susu


### Labels

In [6]:
# VOICE COMMANDS
voice_cmd_class_names = sorted({r['label'] for r in metadata_records})
voice_cmd_class_count = len(voice_cmd_class_names)
voice_cmd_class_id_by_name = {c:i for i, c in enumerate(voice_cmd_class_names)}

print("Classes - Voice Commands")
_ = [print(f"{v:4}: {k}") for k,v in voice_cmd_class_id_by_name.items()]

print("---------------------")


# VOICE COMMAND LANGUAGES
voice_cmd_lng_class_names = sorted({r['language'] for r in metadata_records})
voice_cmd_lng_class_count = len(voice_cmd_lng_class_names)
voice_cmd_lng_class_id_by_name = {c:i for i, c in enumerate(voice_cmd_lng_class_names)}

print("Classes - Voice Command Languages")
_ = [print(f"{v:3}: {k}") for k,v in voice_cmd_lng_class_id_by_name.items()]

print("---------------------")



# SPEAKER MOTHERTONGUE
spkr_mothertongue_class_names = sorted({r['speaker_mothertongue'] for r in metadata_records})
spkr_mothertongue_class_count = len(spkr_mothertongue_class_names)
spkr_mothertongue_class_id_by_name = {c:i for i,c in enumerate(spkr_mothertongue_class_names)}

print("Classes - Speaker Mothertongues")
_ = [print(f"{v:3}: {k}") for k,v in spkr_mothertongue_class_id_by_name.items()]

print("---------------------")



# SPEAKER GENDER
spkr_gender_class_names = sorted({r['speaker_gender'] for r in metadata_records})
spkr_gender_class_count = len(spkr_gender_class_names)
spkr_gender_class_id_by_name = {c:i for i, c in enumerate(spkr_gender_class_names)}

print("Classes - Speaker Gender")
_ = [print(f"{v:3}: {k}") for k,v in spkr_gender_class_id_by_name.items()]

print("----------------------")



Classes - Voice Commands
   0: 101_wake_word__francais
   1: 101_wake_word__maninka
   2: 101_wake_word__pular
   3: 101_wake_word__susu
   4: 201_add_contact__francais
   5: 201_add_contact__maninka
   6: 201_add_contact__pular
   7: 201_add_contact__susu
   8: 202_search_contact__francais
   9: 202_search_contact__maninka
  10: 202_search_contact__pular
  11: 202_search_contact__susu
  12: 203_update_contact__francais
  13: 203_update_contact__maninka
  14: 203_update_contact__pular
  15: 203_update_contact__susu
  16: 204_delete_contact__francais
  17: 204_delete_contact__maninka
  18: 204_delete_contact__pular
  19: 204_delete_contact__susu
  20: 205_call_contact__francais
  21: 205_call_contact__maninka
  22: 205_call_contact__pular
  23: 205_call_contact__susu
  24: 206_yes__francais
  25: 206_yes__maninka
  26: 206_yes__pular
  27: 206_yes__susu
  28: 207_no__francais
  29: 207_no__maninka
  30: 207_no__pular
  31: 207_no__susu
  32: 301_zero__francais
  33: 301_zero__maninka
  

### Inspect metadata

In [7]:
def count_by_attribute(records, attribute_names):
    attribute_name_instances = {}
    for attribute_name in attribute_names:
        attribute_name_instances[attribute_name] = {r[attribute_name] for r in records}
        
    l = [attribute_name_instances[attribute_name] for attribute_name in attribute_names]
    
    
    
    for attribute_values in sorted(itertools.product(*l)):
        
        def record_match(r):
            for i in range(len(attribute_names)):
                if r[attribute_names[i]] != attribute_values[i]:
                    return False
            return True
            
        record_instances = [r for r in records if record_match(r)]
        count = len(record_instances)
        
        yield (attribute_values, count)

In [8]:
print("RECORDS BY DEVICE")
_ = [print(f"\t{r}") for r in sorted(count_by_attribute(metadata_records, ['device_id']))]
print("")

print("RECORDS BY LANGUAGE")
_ = [print(f"\t{r}") for r in sorted(count_by_attribute(metadata_records, ['language']))]
print("")

print("RECORDS BY GENDER")
_ = [print(f"\t{r}") for r in sorted(count_by_attribute(metadata_records, ['speaker_gender']))]
print("")

print("RECORDS BY AGE")
_ = [print(f"\t{r}") for r in sorted(count_by_attribute(metadata_records, ['speaker_age']))]
print("")

print("RECORDS BY SPEAKER")
_ = [print(f"\t{r}") for r in sorted(count_by_attribute(metadata_records, ['speaker_id']))]
print("")

print("RECORDS BY SPEAKER BY LANGUAGE")
_ = [print(f"\t{r}") for r in sorted(count_by_attribute(metadata_records, ['speaker_id', 'language']))]
print("")

print("RECORDS BY SPEAKER BY LABEL")
_ = [print(f"\t{r}") for r in sorted(count_by_attribute(metadata_records, ['label']))]
print("")

RECORDS BY DEVICE
	(('d001',), 2759)
	(('d002',), 2741)
	(('d003',), 2759)

RECORDS BY LANGUAGE
	(('_language_independent',), 3072)
	(('francais',), 1260)
	(('maninka',), 1356)
	(('pular',), 909)
	(('susu',), 1662)

RECORDS BY GENDER
	(('F',), 2820)
	(('M',), 5439)

RECORDS BY AGE
	(('12',), 237)
	(('13',), 252)
	(('15',), 603)
	(('17',), 1050)
	(('18',), 855)
	(('19',), 273)
	(('20',), 291)
	(('27',), 255)
	(('28',), 183)
	(('29',), 237)
	(('31',), 255)
	(('32',), 291)
	(('33',), 183)
	(('34',), 129)
	(('35',), 498)
	(('37',), 309)
	(('38',), 441)
	(('43',), 183)
	(('44',), 540)
	(('5',), 129)
	(('55',), 183)
	(('61',), 390)
	(('63',), 237)
	(('67',), 255)

RECORDS BY SPEAKER
	(('s001',), 183)
	(('s002',), 129)
	(('s003',), 183)
	(('s004',), 183)
	(('s005',), 129)
	(('s006',), 129)
	(('s007',), 183)
	(('s008',), 129)
	(('s009',), 237)
	(('s010',), 291)
	(('s011',), 129)
	(('s012',), 183)
	(('s013',), 129)
	(('s014',), 237)
	(('s015',), 291)
	(('s016',), 183)
	(('s017',), 237)
	(('s018

## Prepare Cross Validation Folds
- Partition by (speaker, language)
- Each (speaker, language) correspond to `utterance_count * device_count`
- For each fold, all `utterance_count * device_count` records for the same speaker in the same language are either in the TRAIN or the VALIDATION sets, but not both.


In [9]:
def generate_train_test_records_per_fold(all_records):
    records_per_fold = {}
    
    all_speaker_languages = sorted({(r['speaker_id'], r['language']) for r in all_records})

    sl_count = len(all_speaker_languages)
    all_sl_indices = range(sl_count)
    train_sl_count = int(np.ceil(sl_count*TRAIN_PERCENT))
    test_sl_count = sl_count - train_sl_count

    for fold_index in range(FOLD_COUNT):
        fold_rsampler = np.random.RandomState(seed=fold_index)

        train_sl_index_set = set(fold_rsampler.choice(all_sl_indices, train_sl_count, replace=False))
        train_sl_set = {all_speaker_languages[i] for i in train_sl_index_set}

        test_sl_index_set = set(all_sl_indices).difference(train_sl_index_set)
        test_sl_set = {all_speaker_languages[i] for i in test_sl_index_set}

        train_records = [r for r in all_records if (r['speaker_id'], r['language']) in train_sl_set]
        test_records = [r for r in all_records if (r['speaker_id'], r['language']) in test_sl_set]
        
        
        records_per_fold[fold_index] = {
            "train_records": train_records,
            "test_records": test_records
        }
    
    return records_per_fold

In [10]:
records_per_fold = generate_train_test_records_per_fold(metadata_records)

### Inspect Folds

In [11]:
for fold_index in range(FOLD_COUNT):
    train_records = records_per_fold[fold_index]["train_records"]
    test_records = records_per_fold[fold_index]["test_records"]

    print(f"Fold {fold_index} -- TRAIN")
    _ = [print(f"\t{r}") for r in sorted(count_by_attribute(train_records, ['speaker_id', 'language'])) if r[1]>0]

    print(f"Fold {fold_index} -- TEST")
    _ = [print(f"\t{r}") for r in sorted(count_by_attribute(test_records, ['speaker_id', 'language'])) if r[1]>0]
    
    print("---------------------")

Fold 0 -- TRAIN
	(('s001', '_language_independent'), 75)
	(('s001', 'maninka'), 54)
	(('s001', 'susu'), 54)
	(('s002', '_language_independent'), 75)
	(('s002', 'maninka'), 54)
	(('s003', '_language_independent'), 75)
	(('s003', 'maninka'), 54)
	(('s003', 'susu'), 54)
	(('s004', '_language_independent'), 75)
	(('s004', 'susu'), 54)
	(('s005', '_language_independent'), 75)
	(('s006', '_language_independent'), 75)
	(('s006', 'maninka'), 54)
	(('s007', '_language_independent'), 75)
	(('s007', 'pular'), 54)
	(('s007', 'susu'), 54)
	(('s008', '_language_independent'), 75)
	(('s008', 'maninka'), 54)
	(('s009', 'pular'), 54)
	(('s009', 'susu'), 54)
	(('s010', '_language_independent'), 75)
	(('s010', 'maninka'), 54)
	(('s010', 'pular'), 54)
	(('s010', 'susu'), 54)
	(('s011', '_language_independent'), 75)
	(('s011', 'susu'), 54)
	(('s012', '_language_independent'), 75)
	(('s012', 'francais'), 54)
	(('s012', 'susu'), 54)
	(('s013', '_language_independent'), 75)
	(('s013', 'susu'), 54)
	(('s014', 

	(('s002', 'maninka'), 54)
	(('s003', '_language_independent'), 75)
	(('s003', 'maninka'), 54)
	(('s004', '_language_independent'), 75)
	(('s004', 'maninka'), 54)
	(('s005', '_language_independent'), 75)
	(('s005', 'susu'), 54)
	(('s006', '_language_independent'), 75)
	(('s007', '_language_independent'), 75)
	(('s007', 'pular'), 54)
	(('s007', 'susu'), 54)
	(('s008', '_language_independent'), 75)
	(('s009', 'susu'), 54)
	(('s010', 'francais'), 54)
	(('s010', 'pular'), 54)
	(('s010', 'susu'), 54)
	(('s011', 'susu'), 54)
	(('s012', '_language_independent'), 75)
	(('s012', 'francais'), 54)
	(('s012', 'susu'), 54)
	(('s013', '_language_independent'), 75)
	(('s013', 'susu'), 54)
	(('s014', '_language_independent'), 75)
	(('s014', 'francais'), 54)
	(('s015', '_language_independent'), 75)
	(('s015', 'maninka'), 54)
	(('s015', 'pular'), 54)
	(('s016', '_language_independent'), 75)
	(('s016', 'maninka'), 54)
	(('s016', 'susu'), 54)
	(('s017', '_language_independent'), 75)
	(('s017', 'francais')

In [12]:
for fold_index in range(FOLD_COUNT):
    train_records = records_per_fold[fold_index]["train_records"]
    test_records = records_per_fold[fold_index]["test_records"]

    print(f"Fold {fold_index} -- TRAIN: ({len(train_records)})")
    _ = [print(f"\t{r}") for r in sorted(count_by_attribute(train_records, ['language'])) if r[1]>0]

    print(f"Fold {fold_index} -- TEST: ({len(test_records)})")
    _ = [print(f"\t{r}") for r in sorted(count_by_attribute(test_records, ['language'])) if r[1]>0]
    
    print("---------------------")

Fold 0 -- TRAIN: (6018)
	(('_language_independent',), 2400)
	(('francais',), 738)
	(('maninka',), 960)
	(('pular',), 852)
	(('susu',), 1068)
Fold 0 -- TEST: (2241)
	(('_language_independent',), 672)
	(('francais',), 522)
	(('maninka',), 396)
	(('pular',), 57)
	(('susu',), 594)
---------------------
Fold 1 -- TRAIN: (5940)
	(('_language_independent',), 2325)
	(('francais',), 978)
	(('maninka',), 906)
	(('pular',), 519)
	(('susu',), 1212)
Fold 1 -- TEST: (2319)
	(('_language_independent',), 747)
	(('francais',), 282)
	(('maninka',), 450)
	(('pular',), 390)
	(('susu',), 450)
---------------------
Fold 2 -- TRAIN: (6036)
	(('_language_independent',), 2397)
	(('francais',), 972)
	(('maninka',), 912)
	(('pular',), 573)
	(('susu',), 1182)
Fold 2 -- TEST: (2223)
	(('_language_independent',), 675)
	(('francais',), 288)
	(('maninka',), 444)
	(('pular',), 336)
	(('susu',), 480)
---------------------
Fold 3 -- TRAIN: (5796)
	(('_language_independent',), 2097)
	(('francais',), 918)
	(('maninka',), 

## Load Features

In [13]:
def load_features(records, feature_name):
    features_list = []
    
    features_input_dir = BASE_DIR / feature_name

    for r in records:
        feature_file_name = r['file'].replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            
            padded_features = np.zeros((MAX_FEATURE_SEQUENCE_LENGTH, 512), dtype=features.dtype)
            padded_features[:features_shape[0], :] = features
            
            
            features_list.append(padded_features)
    return features_list

In [14]:
def get_bias_category_labels(records):
    bias_category_labels = {}
    
    for cat in bias_categories:
        for cat_val in bias_categories[cat]:
            bias_category_labels[f"{cat}__{cat_val}"] = [1 if r[cat]==cat_val else 0 for r in records]
            
    return bias_category_labels

# Classification Models

In [15]:
class ASRCNN(nn.Module):
    def __init__(self, 
                 conv_pooling_type, 
                 conv_dropout_p, 
                 fc_dropout_p, 
                 voice_cmd_neuron_count, 
                 voice_cmd_lng_neuron_count,
                 objective_type
                ):
        
        super(ASRCNN, self).__init__()
        
        if conv_pooling_type not in {"max", "avg"}:
            raise ValueError(f"Unknown Conv Pooling Type: {conv_pooling_type}")
            
        conv_pooling_class_by_type = {
            "max": nn.MaxPool1d,
            "avg": nn.AvgPool1d,
        }
        
        conv_pooling_class = conv_pooling_class_by_type[conv_pooling_type]
        
        self.objective_type = objective_type
        
        self.conv0 = nn.Conv1d(in_channels=512, out_channels=8, kernel_size=1)
        
        self.conv1 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3)
        self.drop1 = nn.Dropout(p=conv_dropout_p)
        self.pool1 = conv_pooling_class(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3)
        self.drop2 = nn.Dropout(p=conv_dropout_p)
        self.pool2 = conv_pooling_class(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
        self.drop3 = nn.Dropout(p=conv_dropout_p)
        self.pool3 = conv_pooling_class(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.drop4 = nn.Dropout(p=conv_dropout_p)
        self.pool4 = conv_pooling_class(kernel_size=2, stride=2)
        
        self.drop5 = nn.Dropout(p=fc_dropout_p)
        
        self.lin61 = nn.Linear(in_features=112, out_features=voice_cmd_neuron_count)
        
        # 'voice_cmd', 'voice_cmd__and__voice_cmd_lng'
        if self.objective_type == 'voice_cmd__and__voice_cmd_lng':
            self.lin62 = nn.Linear(in_features=112, out_features=voice_cmd_lng_neuron_count)
                
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.mean(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.mean(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.mean(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        v = self.drop5(v)
        
        if self.objective_type == 'voice_cmd':
            logits_voice_cmd = self.lin61(v)
            return logits_voice_cmd
        elif self.objective_type == 'voice_cmd__and__voice_cmd_lng':
            logits_voice_cmd = self.lin61(v)
            logits_voice_cmd_lng = self.lin62(v)
            return logits_voice_cmd, logits_voice_cmd_lng
        else:
            raise(f"Unknown objective type: {self.objective_type}")

In [16]:
def get_data_for_fold(fold_id, feature_name):
    
    train_records = records_per_fold[fold_id]["train_records"]
    test_records = records_per_fold[fold_id]["test_records"]
    
    train_features = load_features(train_records, feature_name)
    test_features = load_features(test_records, feature_name)
    
    train_x = np.array(train_features)
    test_x = np.array(test_features)
    
    train_y = {}
    train_y['voice_cmd'] = np.array([voice_cmd_class_id_by_name[r['label']] for r in train_records])
    train_y['voice_cmd_lng'] = np.array([voice_cmd_lng_class_id_by_name[r['language']] for r in train_records])
    train_y['spkr_mothertongue'] = np.array([spkr_mothertongue_class_id_by_name[r['speaker_mothertongue']] for r in train_records])
    train_y['spkr_gender'] = np.array([spkr_gender_class_id_by_name[r['speaker_gender']] for r in train_records])
    
    

    
    test_y = {}
    test_y['voice_cmd'] = np.array([voice_cmd_class_id_by_name[r['label']] for r in test_records])
    test_y['voice_cmd_lng'] = np.array([voice_cmd_lng_class_id_by_name[r['language']] for r in test_records])
    test_y['spkr_mothertongue'] = np.array([spkr_mothertongue_class_id_by_name[r['speaker_mothertongue']] for r in test_records])
    test_y['spkr_gender'] = np.array([spkr_gender_class_id_by_name[r['speaker_gender']] for r in test_records])

    train_bias_category_labels = get_bias_category_labels(train_records)
    test_bias_category_labels = get_bias_category_labels(test_records)
    
    return train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels = \
        get_data_for_fold(fold_id, feature_name)
    
    
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y['voice_cmd']),
        torch.tensor(train_y['voice_cmd_lng']),
        # torch.tensor(train_y['spkr_mothertongue']),
        # torch.tensor(train_y['spkr_gender']),
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y['voice_cmd']),
        torch.tensor(test_y['voice_cmd_lng']),
        # torch.tensor(test_y['spkr_mothertongue']),
        # torch.tensor(test_y['spkr_gender']),
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_bias_category_labels, test_bias_category_labels


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [17]:
def train(model, optimizer, criterion, objective_type, train_loader):
    model.train()
    train_loss = 0

    for batch_idx, (x, y_voice_cmd, y_voice_cmd_lng) in enumerate(train_loader):
        x = x.to(device)
        y_voice_cmd = y_voice_cmd.to(device)
        y_voice_cmd_lng = y_voice_cmd_lng.to(device)

        optimizer.zero_grad()
        outputs = model(x)

        if objective_type == 'voice_cmd':
            logits_voice_cmd = outputs
            loss = criterion(logits_voice_cmd, y_voice_cmd)
        elif objective_type == 'voice_cmd__and__voice_cmd_lng':
            logits_voice_cmd, logits_voice_cmd_lng = outputs    
            loss = (criterion(logits_voice_cmd, y_voice_cmd) + criterion(logits_voice_cmd_lng, y_voice_cmd_lng)) / 2
            
        else:
            raise ValueError(f"Unknown objective type: {objective_type}")

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        

def test(model, criterion, objective_type, loader, bias_category_labels):
    model.eval()
    accumulated_loss = 0

    pred_classes = []
    true_classes = []

    pred_classes_lng = []
    true_classes_lng = []

    for batch_idx, (x, y_voice_cmd, y_voice_cmd_lng) in enumerate(loader):
        x = x.to(device)
        y_voice_cmd = y_voice_cmd.to(device)
        y_voice_cmd_lng = y_voice_cmd_lng.to(device)

        outputs = model(x)

        if objective_type == 'voice_cmd':
            logits_voice_cmd = outputs

            pred_classes.extend(
                get_predictions_for_logits(logits_voice_cmd).cpu().numpy()
            )
            true_classes.extend(y_voice_cmd.cpu().numpy())

            loss = criterion(logits_voice_cmd, y_voice_cmd)
        elif objective_type == 'voice_cmd__and__voice_cmd_lng':
            logits_voice_cmd, logits_voice_cmd_lng = outputs
            pred_classes.extend(
                get_predictions_for_logits(logits_voice_cmd).cpu().numpy()
            )
            true_classes.extend(y_voice_cmd.cpu().numpy())

            pred_classes_lng.extend(
                get_predictions_for_logits(logits_voice_cmd_lng).cpu().numpy()
            )
            true_classes_lng.extend(y_voice_cmd_lng.cpu().numpy())

            loss = (criterion(logits_voice_cmd, y_voice_cmd) + criterion(logits_voice_cmd_lng, y_voice_cmd_lng)) /2
        else:
            raise ValueError(f"Unknown objective type: {objective_type}")

        accumulated_loss += loss.item()

    n = len(true_classes)

    average_loss = accumulated_loss/n
    
    acc = sklearn.metrics.accuracy_score(true_classes, pred_classes)
    acc_by_bais_category = {
        category: sklearn.metrics.accuracy_score(true_classes, pred_classes, sample_weight=sw)
        for category, sw in bias_category_labels.items()
    }
    
    
    if objective_type == 'voice_cmd__and__voice_cmd_lng':
        acc_lng = sklearn.metrics.accuracy_score(true_classes_lng, pred_classes_lng)
        acc_by_bais_category_lng = {
            category: sklearn.metrics.accuracy_score(true_classes_lng, pred_classes_lng, sample_weight=sw)
            for category, sw in bias_category_labels.items()
        }
    else:
        acc_lng = -1
        acc_by_bais_category_lng = {
            category: -1
            for category, sw in bias_category_labels.items()
        }
        
    return n, average_loss, acc, acc_by_bais_category, acc_lng, acc_by_bais_category_lng
      
        
def train_on_fold(model, fold_id, feature_name, objective_type, batch_size, epochs):
    torch.manual_seed(0)
    results = {}
    
    train_loader, test_loader, train_bias_category_labels, test_bias_category_labels = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(model, torch.zeros((10, MAX_FEATURE_SEQUENCE_LENGTH, 512)).to(device), show_input=False))
    print(f"train_n: {len(train_loader.dataset)}")
    print(f"test_n: {len(test_loader.dataset)}")

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(reduction='sum')

    for epoch in range(1, epochs+1):
        
        # train on training set
        train(model, optimizer, criterion, objective_type, train_loader)
        
        # test on training set
        train_n, train_average_loss, train_acc, train_acc_by_bais_category, train_acc_lng, train_acc_by_bais_category_lng = \
            test(model, criterion, objective_type, train_loader, train_bias_category_labels)
        
        # test on test set
        test_n, test_average_loss, test_acc, test_acc_by_bais_category, test_acc_lng, test_acc_by_bais_category_lng = \
            test(model, criterion, objective_type, test_loader, test_bias_category_labels)
        

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_average_loss:0.4}. Test Loss: {test_average_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
        
         
        results[epoch] = {
            'epoch': epoch,
            
            'train_n': train_n,
            'train_loss': train_average_loss,
            'train_acc': train_acc,
            'train_acc_lng': train_acc_lng,
            
            'test_n': test_n,
            'test_loss': test_average_loss,
            'test_acc': test_acc,
            'test_acc_lng': test_acc_lng
        }
        
        for c in train_acc_by_bais_category:
            results[epoch][f"train_acc_{c}"] = train_acc_by_bais_category[c]
            results[epoch][f"train_n_{c}"] = int(np.sum(train_bias_category_labels[c]))
            
        for c in train_acc_by_bais_category_lng:
            results[epoch][f"train_acc_lng_{c}"] = train_acc_by_bais_category_lng[c]
            
            
        for c in test_acc_by_bais_category:
            results[epoch][f"test_acc_{c}"] = test_acc_by_bais_category[c]
            results[epoch][f"test_n_{c}"] = int(np.sum(test_bias_category_labels[c]))

        for c in test_acc_by_bais_category_lng:
            results[epoch][f"test_acc_lng_{c}"] = test_acc_by_bais_category_lng[c]
            

    return results

In [18]:
import csv
from pathlib import Path

def results_exist(model_name, feature_name, fold_id):
    fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_id}.csv"
    return Path(fname).is_file()
    
    

def save_results(model_name, all_folds_results):
    for result_entry in all_folds_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        
        Path(RESULTS_DIR).mkdir(exist_ok=True, parents=True)
        fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_index}.csv"
        Path(fname).parent.mkdir(parents=True, exist_ok=True)
        with open(fname, 'w') as f:
            fieldnames = sorted(result_entry['epochs'][1].keys())
            
            writer = csv.DictWriter(f, fieldnames=fieldnames, extrasaction='raise')
            
            writer.writeheader()
            
            for epoch in sorted(result_entry['epochs'].keys()):
                writer.writerow(result_entry['epochs'][epoch])

In [ ]:
trial_params = list(
    itertools.product(
        range(FOLD_COUNT),
        CONV_POOLING_TYPES,
        CONV_DROPOUT_PROBABILITIES,
        FC_DROPOUT_PROBABILITIES,
        FEATURE_NAMES, 
        OBJECTIVE_TYPES
    )
)    

print("Plan:")
_ = [print(t) for t in trial_params]
print()

for fold_id, conv_pooling_type, conv_dropout_p, fc_dropout_p, feature_name, objective_type in trial_params:

    model_name = f"ASRCNN" + \
    f"__conv_pool_{conv_pooling_type}" + \
    f"__conv_dp_{conv_dropout_p}" + \
    f"__fc_dp_{fc_dropout_p}" + \
    f"__fn_{feature_name}" + \
    f"__obj_{objective_type}"
    
    if results_exist(model_name, feature_name, fold_id):
        print(f"skipping ({fold_id}, {conv_pooling_type}, {conv_dropout_p}, {fc_dropout_p}, {feature_name}, {objective_type})")
        continue
    
    model = ASRCNN(
        conv_pooling_type, 
        conv_dropout_p, 
        fc_dropout_p, 
        voice_cmd_neuron_count = voice_cmd_class_count, 
        voice_cmd_lng_neuron_count = voice_cmd_lng_class_count,
        objective_type = objective_type
    ).to(device)

    print(f"{model_name} using {feature_name} on fold#{fold_id}")

    epochs_results = train_on_fold(
        model, 
        fold_id, 
        feature_name, 
        objective_type, 
        batch_size = BATCH_SIZE, 
        epochs = EPOCHS
    )

    # results for only one fold
    folds_results = [{
        'fold_index': fold_id,
        'feature_name': feature_name,
        'epochs': epochs_results
    }]
    
    
    save_results(model_name, folds_results)
    # write_epoch_test_logits(model_name, all_folds_results)

    del model

Plan:
(0, 'avg', 0.1, 0.1, 'wav2vec_features-c', 'voice_cmd')
(0, 'avg', 0.1, 0.1, 'wav2vec_features-c', 'voice_cmd__and__voice_cmd_lng')
(0, 'avg', 0.1, 0.1, 'wav2vec_features-z', 'voice_cmd')
(0, 'avg', 0.1, 0.1, 'wav2vec_features-z', 'voice_cmd__and__voice_cmd_lng')
(0, 'avg', 0.1, 0.1, 'retrained-wav2vec_features-c', 'voice_cmd')
(0, 'avg', 0.1, 0.1, 'retrained-wav2vec_features-c', 'voice_cmd__and__voice_cmd_lng')
(0, 'avg', 0.1, 0.1, 'retrained-wav2vec_features-z', 'voice_cmd')
(0, 'avg', 0.1, 0.1, 'retrained-wav2vec_features-z', 'voice_cmd__and__voice_cmd_lng')
(1, 'avg', 0.1, 0.1, 'wav2vec_features-c', 'voice_cmd')
(1, 'avg', 0.1, 0.1, 'wav2vec_features-c', 'voice_cmd__and__voice_cmd_lng')
(1, 'avg', 0.1, 0.1, 'wav2vec_features-z', 'voice_cmd')
(1, 'avg', 0.1, 0.1, 'wav2vec_features-z', 'voice_cmd__and__voice_cmd_lng')
(1, 'avg', 0.1, 0.1, 'retrained-wav2vec_features-c', 'voice_cmd')
(1, 'avg', 0.1, 0.1, 'retrained-wav2vec_features-c', 'voice_cmd__and__voice_cmd_lng')
(1, 'avg',

Epoch: 460. Train Loss: 0.06146. Test Loss: 1.125. Train Acc: 0.9867. Test Acc:0.7796
Epoch: 470. Train Loss: 0.05545. Test Loss: 1.136. Train Acc: 0.9894. Test Acc:0.7764
Epoch: 480. Train Loss: 0.05112. Test Loss: 1.151. Train Acc: 0.9894. Test Acc:0.7813
Epoch: 490. Train Loss: 0.04693. Test Loss: 1.146. Train Acc: 0.9915. Test Acc:0.7822
Epoch: 500. Train Loss: 0.0486. Test Loss: 1.172. Train Acc: 0.9907. Test Acc:0.7796
Epoch: 510. Train Loss: 0.04207. Test Loss: 1.142. Train Acc: 0.9934. Test Acc:0.78
Epoch: 520. Train Loss: 0.04026. Test Loss: 1.186. Train Acc: 0.9922. Test Acc:0.7813
Epoch: 530. Train Loss: 0.0385. Test Loss: 1.175. Train Acc: 0.9937. Test Acc:0.7813
Epoch: 540. Train Loss: 0.03469. Test Loss: 1.187. Train Acc: 0.9942. Test Acc:0.7791
Epoch: 550. Train Loss: 0.03437. Test Loss: 1.16. Train Acc: 0.9944. Test Acc:0.7854
Epoch: 560. Train Loss: 0.03132. Test Loss: 1.201. Train Acc: 0.995. Test Acc:0.784
Epoch: 570. Train Loss: 0.02988. Test Loss: 1.197. Train Acc:

Epoch: 220. Train Loss: 0.4185. Test Loss: 0.9584. Train Acc: 0.9207. Test Acc:0.722
Epoch: 230. Train Loss: 0.4018. Test Loss: 0.9662. Train Acc: 0.9209. Test Acc:0.7251
Epoch: 240. Train Loss: 0.381. Test Loss: 0.9364. Train Acc: 0.9262. Test Acc:0.73
Epoch: 250. Train Loss: 0.368. Test Loss: 0.9396. Train Acc: 0.9276. Test Acc:0.7332
Epoch: 260. Train Loss: 0.3497. Test Loss: 0.937. Train Acc: 0.9315. Test Acc:0.7305
Epoch: 270. Train Loss: 0.3354. Test Loss: 0.9361. Train Acc: 0.9349. Test Acc:0.7314
Epoch: 280. Train Loss: 0.3195. Test Loss: 0.9346. Train Acc: 0.9408. Test Acc:0.7372
Epoch: 290. Train Loss: 0.3077. Test Loss: 0.936. Train Acc: 0.9397. Test Acc:0.7381
Epoch: 300. Train Loss: 0.2958. Test Loss: 0.9508. Train Acc: 0.943. Test Acc:0.7363
Epoch: 310. Train Loss: 0.287. Test Loss: 0.9442. Train Acc: 0.9455. Test Acc:0.7394
Epoch: 320. Train Loss: 0.2631. Test Loss: 0.9233. Train Acc: 0.9518. Test Acc:0.7456
Epoch: 330. Train Loss: 0.2507. Test Loss: 0.9314. Train Acc: 0

Epoch: 10. Train Loss: 3.904. Test Loss: 4.011. Train Acc: 0.07527. Test Acc:0.05712
Epoch: 20. Train Loss: 3.271. Test Loss: 3.471. Train Acc: 0.1489. Test Acc:0.1156
Epoch: 30. Train Loss: 2.998. Test Loss: 3.242. Train Acc: 0.2275. Test Acc:0.1794
Epoch: 40. Train Loss: 2.645. Test Loss: 2.951. Train Acc: 0.3084. Test Acc:0.2477
Epoch: 50. Train Loss: 2.312. Test Loss: 2.669. Train Acc: 0.3933. Test Acc:0.3097
Epoch: 60. Train Loss: 2.021. Test Loss: 2.439. Train Acc: 0.4653. Test Acc:0.3677
Epoch: 70. Train Loss: 1.785. Test Loss: 2.241. Train Acc: 0.5221. Test Acc:0.4132
Epoch: 80. Train Loss: 1.59. Test Loss: 2.066. Train Acc: 0.5809. Test Acc:0.4645
Epoch: 90. Train Loss: 1.434. Test Loss: 1.932. Train Acc: 0.6183. Test Acc:0.5025
Epoch: 100. Train Loss: 1.303. Test Loss: 1.807. Train Acc: 0.6607. Test Acc:0.5301
Epoch: 110. Train Loss: 1.179. Test Loss: 1.712. Train Acc: 0.6947. Test Acc:0.5591
Epoch: 120. Train Loss: 1.081. Test Loss: 1.617. Train Acc: 0.724. Test Acc:0.5877
E

Epoch: 980. Train Loss: 0.02451. Test Loss: 1.391. Train Acc: 0.9962. Test Acc:0.7702
Epoch: 990. Train Loss: 0.0232. Test Loss: 1.403. Train Acc: 0.9963. Test Acc:0.7711
Epoch: 1000. Train Loss: 0.03135. Test Loss: 1.362. Train Acc: 0.995. Test Acc:0.776
ASRCNN__conv_pool_avg__conv_dp_0.1__fc_dp_0.1__fn_wav2vec_features-z__obj_voice_cmd__and__voice_cmd_lng using wav2vec_features-z on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1        [10, 8, 200]           4,104           4,104
          Conv1d-2        [10, 8, 198]             200             200
         Dropout-3        [10, 8, 198]               0               0
       AvgPool1d-4         [10, 8, 99]               0               0
          Conv1d-5        [10, 16, 97]             400             400
         Dropout-6        [10, 16, 97]               0               0
       AvgPool1d-7        [10, 16, 4

...

Epoch: 80. Train Loss: 1.454. Test Loss: 1.925. Train Acc: 0.62. Test Acc:0.5176
Epoch: 90. Train Loss: 1.264. Test Loss: 1.772. Train Acc: 0.6805. Test Acc:0.5484
Epoch: 100. Train Loss: 1.086. Test Loss: 1.619. Train Acc: 0.7275. Test Acc:0.5832
Epoch: 110. Train Loss: 0.9679. Test Loss: 1.523. Train Acc: 0.7516. Test Acc:0.6069
Epoch: 120. Train Loss: 0.8688. Test Loss: 1.47. Train Acc: 0.7832. Test Acc:0.6274
Epoch: 130. Train Loss: 0.7895. Test Loss: 1.404. Train Acc: 0.8067. Test Acc:0.6493
Epoch: 140. Train Loss: 0.7248. Test Loss: 1.354. Train Acc: 0.8264. Test Acc:0.6577
Epoch: 150. Train Loss: 0.6729. Test Loss: 1.329. Train Acc: 0.8406. Test Acc:0.6747
Epoch: 160. Train Loss: 0.6319. Test Loss: 1.293. Train Acc: 0.8514. Test Acc:0.6814
Epoch: 170. Train Loss: 0.5917. Test Loss: 1.27. Train Acc: 0.8626. Test Acc:0.6867
Epoch: 180. Train Loss: 0.5609. Test Loss: 1.274. Train Acc: 0.8727. Test Acc:0.6979
Epoch: 190. Train Loss: 0.5343. Test Loss: 1.264. Train Acc: 0.875. Test A

Epoch: 10. Train Loss: 2.641. Test Loss: 2.808. Train Acc: 0.08375. Test Acc:0.06426
Epoch: 20. Train Loss: 2.259. Test Loss: 2.472. Train Acc: 0.1834. Test Acc:0.1365
Epoch: 30. Train Loss: 2.015. Test Loss: 2.265. Train Acc: 0.2773. Test Acc:0.2316
Epoch: 40. Train Loss: 1.755. Test Loss: 2.041. Train Acc: 0.3827. Test Acc:0.3061
Epoch: 50. Train Loss: 1.559. Test Loss: 1.885. Train Acc: 0.4721. Test Acc:0.3878
Epoch: 60. Train Loss: 1.4. Test Loss: 1.743. Train Acc: 0.5442. Test Acc:0.4502
Epoch: 70. Train Loss: 1.27. Test Loss: 1.635. Train Acc: 0.5936. Test Acc:0.4922
Epoch: 80. Train Loss: 1.168. Test Loss: 1.554. Train Acc: 0.6394. Test Acc:0.5207
Epoch: 90. Train Loss: 1.087. Test Loss: 1.489. Train Acc: 0.6766. Test Acc:0.5524
Epoch: 100. Train Loss: 1.018. Test Loss: 1.441. Train Acc: 0.7092. Test Acc:0.573
Epoch: 110. Train Loss: 0.9656. Test Loss: 1.413. Train Acc: 0.7318. Test Acc:0.5917
Epoch: 120. Train Loss: 0.9127. Test Loss: 1.372. Train Acc: 0.7522. Test Acc:0.6073
E

Epoch: 980. Train Loss: 0.04951. Test Loss: 1.273. Train Acc: 0.9958. Test Acc:0.743
Epoch: 990. Train Loss: 0.04911. Test Loss: 1.259. Train Acc: 0.996. Test Acc:0.7407
Epoch: 1000. Train Loss: 0.04849. Test Loss: 1.246. Train Acc: 0.9952. Test Acc:0.7456
ASRCNN__conv_pool_avg__conv_dp_0.1__fc_dp_0.1__fn_wav2vec_features-c__obj_voice_cmd using wav2vec_features-c on fold#1
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1        [10, 8, 200]           4,104           4,104
          Conv1d-2        [10, 8, 198]             200             200
         Dropout-3        [10, 8, 198]               0               0
       AvgPool1d-4         [10, 8, 99]               0               0
          Conv1d-5        [10, 16, 97]             400             400
         Dropout-6        [10, 16, 97]               0               0
       AvgPool1d-7        [10, 16, 48]               0 

Epoch: 760. Train Loss: 0.03939. Test Loss: 1.36. Train Acc: 0.9919. Test Acc:0.7585
Epoch: 770. Train Loss: 0.02419. Test Loss: 1.298. Train Acc: 0.996. Test Acc:0.7818
Epoch: 780. Train Loss: 0.02896. Test Loss: 1.318. Train Acc: 0.9939. Test Acc:0.7719
Epoch: 790. Train Loss: 0.02229. Test Loss: 1.301. Train Acc: 0.9965. Test Acc:0.7736
Epoch: 800. Train Loss: 0.0189. Test Loss: 1.324. Train Acc: 0.9981. Test Acc:0.774
Epoch: 810. Train Loss: 0.0195. Test Loss: 1.322. Train Acc: 0.9975. Test Acc:0.7771
Epoch: 820. Train Loss: 0.01861. Test Loss: 1.341. Train Acc: 0.9968. Test Acc:0.7809
Epoch: 830. Train Loss: 0.01938. Test Loss: 1.355. Train Acc: 0.9981. Test Acc:0.7689
Epoch: 840. Train Loss: 0.02174. Test Loss: 1.375. Train Acc: 0.9956. Test Acc:0.7762
Epoch: 850. Train Loss: 0.01926. Test Loss: 1.348. Train Acc: 0.9966. Test Acc:0.771
Epoch: 860. Train Loss: 0.01675. Test Loss: 1.354. Train Acc: 0.998. Test Acc:0.7788
Epoch: 870. Train Loss: 0.01766. Test Loss: 1.377. Train Acc:

Epoch: 530. Train Loss: 0.113. Test Loss: 1.108. Train Acc: 0.9813. Test Acc:0.7572
Epoch: 540. Train Loss: 0.1096. Test Loss: 1.113. Train Acc: 0.9823. Test Acc:0.7589
Epoch: 550. Train Loss: 0.1089. Test Loss: 1.142. Train Acc: 0.984. Test Acc:0.7495
Epoch: 560. Train Loss: 0.1018. Test Loss: 1.116. Train Acc: 0.9848. Test Acc:0.7542
Epoch: 570. Train Loss: 0.09762. Test Loss: 1.143. Train Acc: 0.986. Test Acc:0.746
Epoch: 580. Train Loss: 0.09885. Test Loss: 1.129. Train Acc: 0.9855. Test Acc:0.7568
Epoch: 590. Train Loss: 0.09265. Test Loss: 1.132. Train Acc: 0.9869. Test Acc:0.7525
Epoch: 600. Train Loss: 0.08848. Test Loss: 1.118. Train Acc: 0.9882. Test Acc:0.7589
Epoch: 610. Train Loss: 0.08374. Test Loss: 1.149. Train Acc: 0.9899. Test Acc:0.7577
Epoch: 620. Train Loss: 0.08294. Test Loss: 1.136. Train Acc: 0.9884. Test Acc:0.7529
Epoch: 630. Train Loss: 0.07961. Test Loss: 1.155. Train Acc: 0.9874. Test Acc:0.7546
Epoch: 640. Train Loss: 0.07809. Test Loss: 1.135. Train Acc: 